In [7]:
import os
import sys

sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname("../bpepi/bpepi"))))
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname("../bpepi"))))

import bpepi
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch


In [8]:
import networkx as nx
import numpy as np

In [9]:
import sys
sys.path.insert(0,'../Analysis')
sys.path.insert(0,'../bpepi')

In [10]:
from gen import simulate_one_detSIR
from bpepi.Modules import fg_torch as fg

In [11]:
torch.set_num_threads(2)

## Random Regular Graph

In [12]:
N=2000
d=3
lam=0.8
delta=0.01
rho=0.01
G=nx.random_regular_graph(d,N)
for (u,v) in G.edges():
    G.edges[u,v]['lambda'] = lam
if (N<500) : nx.draw(G)

In [13]:
snn = np.array(simulate_one_detSIR(G, s_type = "delta", S = delta, mask = ["SI"], T_max=100))

In [14]:
T = len(snn) - 1

In [15]:
from gen import generate_contacts
contacts = generate_contacts(G,T,lam)

#### Observations

In [16]:
from gen import generate_sensors_obs
num_obs = int(rho*N)
obs = generate_sensors_obs(snn, o_type="n_obs", M=num_obs)

In [17]:
tol=1e-4
damp=0.

In [27]:
def print_iter(err, it):
    print(f"\r err:{err[0]:.2}, err:{err[1]:.2}, it:{it}", end="")

In [28]:
import torch
contacts_t = torch.tensor(contacts)

In [50]:
from bpepi.Modules import st_torch as st_p
from bpepi.Modules import st as st
import importlib
importlib.reload(st_p)
importlib.reload(st)

from bpepi.Modules import fg_torch as fg_p
from bpepi.Modules import fg as fg
import importlib
importlib.reload(fg)
importlib.reload(fg_p)

dtype=torch.float32
device="cpu"
fg_p_ = fg_p.FactorGraph(N,T,contacts,obs,delta=delta, dtype=dtype, device=device)
fg_ = fg.FactorGraph(N,T,contacts,obs,delta=delta)

In [41]:
%timeit fg_.iterate(damp=damp)

21.4 ms ± 731 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [42]:
%timeit fg_p_.iterate(damp=damp)

11.6 ms ± 294 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [47]:
fg_p_ = fg_p.FactorGraph(N,T,contacts,obs,delta=delta, dtype=dtype, device=device)
fg_ = fg.FactorGraph(N,T,contacts,obs,delta=delta)

In [48]:
fg_.update(tol=tol, print_iter=print_iter, damp=damp, maxit=1000)

 err:0.027, err:8.7e-05, it:19

(19, [0.027422351270527223, 8.690153412090338e-05])

In [51]:
fg_p_.update(tol=tol, print_iter=print_iter, damp=damp, maxit=1000)

 err:0.027, err:8.7e-05, it:19

(19, [tensor(0.0274), tensor(8.6900e-05)])

In [ ]:
(np.abs(fg_p_.messages.values.numpy() - fg_.messages.values)).mean()

2.1923592325971343e-08

In [ ]:
marg = fg_.marginals()
marg_p = fg_p_.marginals().numpy()

In [ ]:
(np.abs(marg - marg_p)).mean()

2.334497065292663e-07